In [1]:
import random
import xxhash
from implementations.hyperloglog import HyperLogLog
from implementations.recordinality import Recordinality
from implementations.zipfian_data_generator import ZipfianDataGenerator

In [2]:
random.seed(0xCAFFE) # for the sake of reproducibility

In [3]:
def xxhash64(s: str):
    return xxhash.xxh64(s).intdigest()

In [4]:
def stream_from_file(filename: str):
    stream = []
    with open(filename, 'r') as file:
        for line in file:
            word = line.strip()
            stream.append(word)
    return stream

In [5]:
def stream_cardinality_estimation(stream: list[str], cardinality_estimator):
    ground_truth_count = len(set(stream))
    for word in stream:
        cardinality_estimator.add(word)
    estimation = cardinality_estimator.count()
    relative_error = abs(ground_truth_count - estimation) / ground_truth_count
    return ground_truth_count, estimation, relative_error


In [6]:
HLL = HyperLogLog(hash_f=xxhash64, hash_size=64, m=2**8)
HLL.count()

367.7555677437675

In [7]:
max(HLL.hash_leading_zeros(HLL.hash_f(str(4534543345+i)) & HLL.mask) for i in range(2**16))

15

In [8]:
HLL.hash_leading_zeros(2**2-1)

54

In [9]:
HLL.mask_len

56

In [10]:
ZDG = ZipfianDataGenerator(1, 10)
ZDG.random_stream(10)

['jtvqqh',
 'szidqw',
 'jtvqqh',
 'jujifg',
 'xjsspy',
 'xjsspy',
 'jtvqqh',
 'jtvqqh',
 'jtvqqh',
 'jtvqqh']

In [11]:
%%time
HLL = HyperLogLog(hash_f=xxhash64, hash_size=64, m=2**8)
stream = stream_from_file("datasets/quijote.txt")
stream_cardinality_estimation(stream, HLL)

CPU times: user 148 ms, sys: 12 ms, total: 160 ms
Wall time: 191 ms


(23034, 24254.286700800636, 0.05297762875751654)

In [12]:
N = 10**5
stream = ZipfianDataGenerator(alpha=1, n=N).random_stream(4*N)
HLL = HyperLogLog(hash_f=xxhash64, hash_size=64, m=2**8)
stream_cardinality_estimation(stream, HLL)

(55780, 57056.46166658794, 0.022883859207385025)

In [15]:
%%time
R = Recordinality(hash_f=xxhash64, k=2**8)
stream = stream_from_file("datasets/quijote.txt")
stream_cardinality_estimation(stream, R)

CPU times: user 167 ms, sys: 19 µs, total: 167 ms
Wall time: 167 ms


(23034, 24985.224522334935, 0.0847106243958902)

In [20]:
N = 10**5
stream = ZipfianDataGenerator(alpha=1, n=N).random_stream(4*N)
R = Recordinality(hash_f=xxhash64, k=2**8)
stream_cardinality_estimation(stream, R)

(55675, 57998.30244575456, 0.04172972511458575)

In [14]:
R